

Referenced　notebooks:
https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial
https://www.kaggle.com/stefanschulmeister87/all-properties-and-methods-on-one-page



In [ ]:
!rm -rf log
!mkdir log

In [ ]:
!node --version

We will also need Kaggle Environments

In [ ]:
!pip install kaggle-environments -U

Next, we have to import the `make` function from the `kaggle_environments` package

In [ ]:
from kaggle_environments import make

## Building from Scratch

The following bit of code is all you need for a empty agent that does nothing

In [ ]:
# run this if using kaggle notebooks
!cp -r ../input/lux-ai-2021/* .

We have something that survives! We are now ready to submit something to the leaderboard. The code below compiles all we have built so far into one file that you can then submit to the competition leaderboard

In [ ]:
%%writefile game_info.py
import numpy as np
from sklearn.cluster import KMeans
from lux.game_map import Position
from lux.constants import Constants
import math

class GameInfo:
    # get clusters by resrouce
    def __init__(self, game_state, observation):
        self.clusters = {}
        player, opponent = self.get_players(game_state, observation)
        self.player = player
        X, self.resource_array = GameInfo.find_resource_map(game_state, player)
        max_clusters = game_state.map_width//3
        min_length = game_state.map_width//5
        (self.kmeans, self.n_clusters) = GameInfo.make_kmeans(max_clusters, X, min_length)
        if self.kmeans is None:
            return
        for label in range(self.n_clusters):
            self.add_label_info(label)
        self.update(game_state, observation)
        
    def add_label_info(self, label):
        self.clusters[label] = {}
        self.clusters[label]['label'] = label
        center = self.kmeans.cluster_centers_[label]
        self.clusters[label]['center'] = Position(round(center[0]), round(center[1]))

    def get_players(self, game_state, observation):
        return (game_state.players[observation.player], game_state.players[(observation.player + 1) % 2])

    def update(self, game_state, observation):
        player, opponent = self.get_players(game_state, observation)
        for label in range(self.n_clusters):
            self.update_resource(label)
        self.add_units('units', player.units)
        self.add_units('opponent_units', opponent.units)
        self.add_citytiles('player_cities', player.cities)
        self.add_citytiles('opponent_cities', opponent.cities)
        
    def get_cluster_label_unit(self, unit, player):
        if (player == self.player):
            key = 'units'
        else:
            key = 'opponent_units'
        for label in range(self.n_clusters):
            if (unit.id in self.clusters[label][key]):
                return label
        return None

    def find_closest_cluster(self, unit, unit_number=None):
        closest_dist = math.inf
        closest_cluster = None
        for label, cluster in self.clusters.items():
            dist = cluster["center"].distance_to(unit.pos)
            if dist < closest_dist and (unit_number is None or len(cluster['units']) <= unit_number):
                closest_dist = dist
                closest_cluster = cluster
        return closest_cluster

    @staticmethod
    def make_kmeans(max_clusters, X, min_distance):
        max_clusters = min(max_clusters,X.size)
        for n_clusters in reversed(range(1, max_clusters)):
            try:
                kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
                positions = [Position(center[0], center[1]) for center in kmeans.cluster_centers_]
                is_near = False
                for p1 in positions:
                    for p2 in positions:
                        if p1 != p2 and p1.distance_to(p2) < min_distance:
                            is_near = True
                if not is_near:
                    return (kmeans, n_clusters)
            except:
                continue
        return (None, 1)

    def add_units(self, key, units):
        X = np.array([[unit.pos.x, unit.pos.y] for unit in units])
        if X.size > 0:
            labels = self.kmeans.predict(X)
            for label in range(self.n_clusters):
                self.clusters[label][key] = {units[index].id: units[index] for index in range(len(labels)) if
                                             labels[index] == label}
        else:
            for label in range(self.n_clusters):
                self.clusters[label][key] = {}

    def add_citytiles(self, key, cities):
        citytiles = [citytile for city in cities.values() for citytile in city.citytiles]
        X = np.array([[citytile.pos.x, citytile.pos.y] for citytile in citytiles])
        if X.size > 0:
            labels = self.kmeans.predict(X)
            for label in range(self.n_clusters):
                self.clusters[label][key] = {
                    citytiles[index].cityid + '_{}_{}'.format(citytiles[index].pos.x, citytiles[index].pos.y):
                        citytiles[index] for index in range(len(labels)) if
                    labels[index] == label}
        else:
            for label in range(self.n_clusters):
                self.clusters[label][key] = {}

    @staticmethod
    def find_resource_map(game_state, player):
        width, height = game_state.map_width, game_state.map_height
        resource_x_y_array = []
        resource_array = []
        for y in range(height):
            for x in range(width):
                cell = game_state.map.get_cell(x, y)
                if cell.has_resource():
                    if cell.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
                    if cell.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
                    resource_x_y_array.append([x, y])
                    resource_array.append(cell)
        return np.array(resource_x_y_array), resource_array


    def update_resource(self, label):
        self.clusters[label]['resources'] = {}
        self.clusters[label]['amount'] = 0
        for index in range(len(self.kmeans.labels_)):
            if self.kmeans.labels_[index] == label and self.resource_array[index].has_resource():
                self.clusters[label]['resources'][
                    str(self.resource_array[index].pos.x) + '_' + str(self.resource_array[index].pos.y)] = \
                    self.resource_array[
                        index]
                self.clusters[label]['amount'] += self.resource_array[index].resource.amount

In [ ]:
%%writefile mission.py
from lux.constants import Constants
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux import annotate
import math


class Mission:
    def __init__(self, mission_func, append_values={}):
        self.mission_func = mission_func
        self.append_values = append_values

    @staticmethod
    def get_max_action(dic_mission, dic_weight, unit, **kwargs):
        dic_weight = sorted(dic_weight.items(), key=lambda x: x[1])
        actions = []
        choice = []
        for action_name, weight in dic_weight:
            action_mission = dic_mission[action_name]
            kwargs.update(action_mission.append_values)
            actions,action,target_position = action_mission.get_action(action_name, unit=unit, **kwargs)
            choice.append(action_name+":"+action)
            if "none" not in action:
                return actions,action,action_name,target_position,choice
        return actions,None,None,None,choice
    
    def get_action(self, action_name, unit, **kwargs):
        (action, target_position) = self.mission_func(unit=unit, **kwargs)
        actions = []
        if "none" not in action:
            actions.append(annotate.sidetext(action_name + ":" + action))
            if target_position is not None and unit.pos != target_position :
                actions.append(annotate.circle(target_position.x, target_position.y))
                actions.append(annotate.line(unit.pos.x, unit.pos.y, target_position.x, target_position.y))
            if "skip" not in action:
                actions.append(action)
        return actions,action,target_position

class Empty():
    def __init__(self, pos):
        self.pos = pos    
    
class Helper:
    @staticmethod
    def str_pos(pos: Position):
        return str(pos.x) + '-' + str(pos.y)

    @staticmethod
    def str_pos_xy(x, y):
        return str(x) + '-' + str(y)

    @staticmethod
    def find_resources(game_state):
        resource_tiles = {}
        width, height = game_state.map_width, game_state.map_height
        for y in range(height):
            for x in range(width):
                cell = game_state.map.get_cell(x, y)
                if cell.has_resource():
                    resource_tiles[Helper.str_pos_xy(x, y)] = cell
        return resource_tiles
    
    @staticmethod    
    def find_unit_tiles(player, check_can_not_act=False, is_worker=None):
        tiles = {}
        for unit in player.units:
            if check_can_not_act and unit.can_act():
                continue
            if is_worker is not None and is_worker == unit.is_worker():
                continue            
            tiles[Helper.str_pos(unit.pos)] = unit
        return tiles
    
    @staticmethod    
    def find_city_tiles(player):
        tiles = {}
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                city_tile.city = city
                tiles[Helper.str_pos(city_tile.pos)] = city_tile
        return tiles
    
    @staticmethod    
    def find_empty_tiles(game_state, player, used_positions):
        empty_tiles = {}
        width, height = game_state.map_width, game_state.map_height
        for y in range(height):
            for x in range(width):
                pos_str = Helper.str_pos_xy(x, y)
                cell = game_state.map.get_cell(x, y)
                empty = True
                if cell.has_resource():
                    empty = False
                if pos_str in Helper.find_city_tiles(player):
                    empty = False
                if pos_str in used_positions:
                    empty = False
                if empty:
                    empty_tiles[pos_str] = Empty(Position(x, y))
        return empty_tiles
    
    @staticmethod    
    def find_closest_city_tile(pos, city_tiles, fuel=None, light_upkeep=None):
        closest_city_tile = None
        closest_dist = math.inf
        for str_pos, city_tile in city_tiles.items():
            dist = city_tile.pos.distance_to(pos)
            if (dist < closest_dist):
                if fuel is not None and fuel < city_tile.city.fuel:
                    continue;
                if light_upkeep is not None and light_upkeep < city_tile.city.light_upkeep:
                    continue;
                closest_dist = dist
                closest_city_tile = city_tile
        return closest_city_tile
    
    @staticmethod    
    def find_closest_tile(pos, tiles):
        closest_dist = math.inf
        closest_tile = None
        for key, tile in tiles.items():
            dist = tile.pos.distance_to(pos)
            if dist < closest_dist:
                closest_dist = dist
                closest_tile = tile
        return closest_tile
    
    @staticmethod    
    def find_closest_resources(pos, player, resource_tiles):
        closest_dist = math.inf
        closest_resource_tile = None
        for key, resource_tile in resource_tiles.items():
            dist = resource_tile.pos.distance_to(pos)
            if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
            if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
            if resource_tile.resource.amount > 0 and dist < closest_dist:
                closest_dist = dist
                closest_resource_tile = resource_tile
        return closest_resource_tile
    
    @staticmethod    
    def calc_move_pos(unit, player, pos, used_positions):
        unit_tiles = Helper.find_unit_tiles(player)
        check_dirs = [
            Constants.DIRECTIONS.NORTH,
            Constants.DIRECTIONS.EAST,
            Constants.DIRECTIONS.SOUTH,
            Constants.DIRECTIONS.WEST,
        ]
        adjacent_pos = [unit.pos.translate(direction,1) for direction in check_dirs]
        min_distance = pos.distance_to(unit.pos)-1
        root_pos = [ adjacent_pos for adjacent_pos in adjacent_pos if adjacent_pos.distance_to(pos) == min_distance]
        not_used_pos = [pos for pos in root_pos if Helper.str_pos(pos) not in used_positions]
        best_pos = [pos for pos in not_used_pos if Helper.str_pos(pos) not in unit_tiles]
        if len(best_pos) > 0:
            return best_pos[0]
        if len(not_used_pos) > 0:
            return not_used_pos[0]
        return None

class Action:
    @staticmethod
    def build_city(unit, player,used_positions,game_state, **kwargs):
        if unit.can_build(game_state.map):
            used_positions[Helper.str_pos(unit.pos)] = unit
            return unit.build_city(), unit.pos
        if (unit.cargo.wood + unit.cargo.coal + unit.cargo.uranium) >= 100:
            return Action.go_to_empty(unit, player, used_positions=used_positions,**kwargs)
        return "none_not_enough_cargo", None
    
    @staticmethod  
    def go_to_city(unit, player, city_tiles, fuel=None, light_upkeep=None, cargo_have=None, **kwargs):
        if (cargo_have is not None
                and cargo_have >= 100 - unit.get_cargo_space_left()):
            return "none_not_enough_cargo", None
        closest_city_tile = Helper.find_closest_city_tile(unit.pos, city_tiles, fuel, light_upkeep)
        if closest_city_tile is not None:
            del city_tiles[Helper.str_pos(closest_city_tile.pos)]
            return  Action.go_to_pos(unit, player, closest_city_tile.pos,**kwargs)
        return "none_not_found", None
  
    @staticmethod  
    def go_to_pos(unit, player, pos, used_positions, **kwargs):
        if pos != unit.pos:
            go_pos = Helper.calc_move_pos(unit, player, pos, used_positions )
            if go_pos is None:
                action = "skip_collided"
            else:
                direction = unit.pos.direction_to(go_pos)
                action = unit.move(direction)
                pos_str = Helper.str_pos(go_pos)
                used_positions[pos_str] = unit
        else:
            action = "skip_here"
    
        if "skip" in action:
            used_positions[Helper.str_pos(unit.pos)] = unit
        return action, pos
    
    @staticmethod  
    def go_to_empty(unit, player, empty_tiles, **kwargs):
        empty_tile = Helper.find_closest_tile(unit.pos, empty_tiles)
        if empty_tile is None:
            return "none_not_found", None
        if unit.pos == empty_tile.pos:
            return "skip_here", None
        del empty_tiles[Helper.str_pos(empty_tile.pos)]
        return  Action.go_to_pos(unit, player, empty_tile.pos, **kwargs)
  
    @staticmethod  
    def go_to_closest_cluster(unit, player, game_info, is_empty=False, **kwargs):
        cluster = game_info.find_closest_cluster(unit, is_empty)
        if cluster is None:
            return "none_not_found", None
        return  Action.go_to_pos(unit, player, cluster["center"], **kwargs)
        
    
    @staticmethod
    def do_pillage(unit, player, opponent_unit_tiles, **kwargs):
        unit_tile = Helper.find_closest_tile(unit.pos, opponent_unit_tiles)
        if unit_tile is None:
            return "none not found", None
        if unit_tile.pos.distance_to(unit.pos) > 1:
            return Action.go_to_pos(unit, player, unit_tile.pos, **kwargs)
        return unit.pillage(), None
  
    @staticmethod  
    def go_to_closest_resource(unit, player, resource_tiles, **kwargs):
        resource_tile = Helper.find_closest_resources(unit.pos, player, resource_tiles)
        return  Action.go_to_resource(unit, player, resource_tile, resource_tiles, **kwargs)
    
    @staticmethod
    def go_to_resource(unit, player, resource_tile, tiles, **kwargs):
        if unit.get_cargo_space_left() > 5:
            if resource_tile is not None:
                del tiles[Helper.str_pos(resource_tile.pos)]
                return  Action.go_to_pos(unit, player, resource_tile.pos, **kwargs)
        return "none_not_found", None
    
    @staticmethod
    def go_to_pos_resource(unit, player, pos, resource_tiles, **kwargs):
        resource_tile = Helper.find_closest_resources(pos, player, resource_tiles)
        return  Action.go_to_resource(unit, player, resource_tile, resource_tiles, **kwargs)


In [ ]:
%%writefile game_logger.py
import pandas as pd
class GameLogger:
    def __init__(self,game_id):
        self.game_id = game_id
        self.turn_logs = {}

    def append_obj(self, turn, uniq_id, obj):
        attr = vars(obj)
        for key, value in attr.items():
            self.append(turn, uniq_id, key, value)
    
    def append(self, turn, uniq_id, key, value):
        dic_key = "{}-{}".format(uniq_id,key)
        if turn not in self.turn_logs:
            self.turn_logs[turn] = {}
        if dic_key not in self.turn_logs[turn]:
            self.turn_logs[turn][dic_key] = {}
        self.turn_logs[turn][dic_key] = value
    
    def get_pandas(self):
        df = pd.DataFrame.from_dict(
            {turn: pd.Series(v) for turn, v in self.turn_logs.items()}, orient='index')
        df = df.reindex(sorted(df.columns), axis=1)
        return df
      
    
    def save(self,team):
        df = self.get_pandas()
        df.to_csv('log/{}-{}.csv'.format(self.game_id,team))
        

In [ ]:
%%writefile agent.py
import math, sys
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import logging
import numpy as np

logging.basicConfig(filename="agent.log", level=logging.INFO)

DIRECTIONS = Constants.DIRECTIONS
game_state = None

build_location = None

unit_to_city_dict = {}

unit_to_resource_dict = {}

def get_resource_tiles(game_state, width, height):
    resource_tiles: list[Cell] = []
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

def get_close_resource(unit, resource_tiles, player):
    closest_dist = math.inf
    closest_resource_tile = None
    # if the unit is a worker and we have space in cargo, lets find the nearest resource tile and try to mine it
    for resource_tile in resource_tiles:
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        if resource_tile in unit_to_resource_dict.values(): continue
        
        dist = resource_tile.pos.distance_to(unit.pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile
            
def get_close_city(player, unit):
    closest_dist = math.inf
    closest_city_tile = None
    for k, city in player.cities.items():
        for city_tile in city.citytiles:
            dist = city_tile.pos.distance_to(unit.pos)
            if dist < closest_dist:
                closest_dist = dist
                closest_city_tile = city_tile
    return closest_city_tile

def find_empty_tile_near(near_what, game_state, observation):
    build_location = None
    
    dirs = [(1,0), (0,1), (-1,0), (0,-1)]
    
    for d in dirs:
        try:
            possible_empty_tile = game_state.map.get_cell(near_what.pos.x+d[0], near_what.pos.y+d[1])
            if possible_empty_tile.resource == None and possible_empty_tile.road == 0 and possible_empty_tile.citytile == None:
                build_location = possible_empty_tile
                return build_location
        except Exception as e:
            pass
        
    dirs = [(1,-1), (-1,1), (-1,-1), (1,1)]
    
    for d in dirs:
        try:
            possible_empty_tile = game_state.map.get_cell(near_what.pos.x+d[0], near_what.pos.y+d[1])
            if possible_empty_tile.resource == None and possible_empty_tile.road == 0 and possible_empty_tile.citytile == None:
                build_location = possible_empty_tile
                return build_location
        except Exception as e:
            pass
    

def agent(observation, configuration):
    global game_state
    global build_location
    global unit_to_city_dict
    global unit_to_resource_dict

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    resource_tiles = get_resource_tiles(game_state, width, height)
    workers = [u for u in player.units if u.is_worker()]
    
    for w in workers:
        if w.id not in unit_to_city_dict:
            pass
            # print(f"{observation['step']} Found worker unaccounted for {w}\n")
        city_assignment = get_close_city(player, w)
        unit_to_city_dict[w.id] = city_assignment
        
    for w in workers:
        if w not in unit_to_resource_dict:
            pass
            # print(f"{observation['step']} Found worker w/o resource {w}\n")
        resource_assignment = get_close_resource(w, resource_tiles, player)
        unit_to_resource_dict[w.id] = resource_assignment
        
    # print(f"{observation['step']} workers {workers}\n")
    
    cities = player.cities.values()
    city_tiles = []
    
    for city in cities:
        for c_tile in city.citytiles:
            city_tiles.append(c_tile)
        
    
    
    #logging.info(f"{cities}")
    #logging.info(f"{city_tiles}")
    
    build_city = False
#     if len(city_tiles) < 2:
#         build_city = True
    

    if len(workers) / len(city_tiles) >= 0.75:
        build_city = True
    
    # we iterate over all our units and do something with them
    
    for unit in player.units:
        if unit.is_worker() and unit.can_act():
            
            if unit.get_cargo_space_left() > 0: 
                intended_resource = unit_to_resource_dict[unit.id]
                cell = game_state.map.get_cell(intended_resource.pos.x, intended_resource.pos.y)
                
                if cell.has_resource():
                    actions.append(unit.move(unit.pos.direction_to(intended_resource.pos)))
                    
                else:
                    intended_resource = get_close_resource(unit, resource_tiles, player)
                    unit_to_resource_dict[unit.id] = resource_assignment
                    actions.append(unit.move(unit.pos.direction_to(intended_resource.pos)))
                    
#                 closest_resource_tile = get_close_resource(unit, resource_tiles, player)        
#                 if closest_resource_tile is not None:
#                     actions.append(unit.move(unit.pos.direction_to(closest_resource_tile.pos)))
            else:
                if build_city:
                
                    associated_city_id = unit_to_city_dict[unit.id].cityid
                    unit_city = [c for c in cities if c.cityid == associated_city_id][0]
                    unit_city_fuel = unit_city.fuel
                    unit_city_size = len(unit_city.citytiles)
                    
                    emough_fuel = (unit_city_fuel/unit_city_size) > 300
                
                    if build_location is None:
                        empty_near = get_close_city(player, unit)
                        build_location = find_empty_tile_near(empty_near, game_state, observation)
                        

                    
                    if unit.pos == build_location.pos:
                        action = unit.build_city()
                        actions.append(action)

                        build_city = False
                        build_location = None
                        continue
                    else:
                        # actions.append(unit.move(unit.pos.direction_to(build_location.pos)))
                        dir_diff = (build_location.pos.x - unit.pos.x, build_location.pos.y - unit.pos.y)
                        xdiff = dir_diff[0]
                        ydiff = dir_diff[1]
                        
                        if abs(ydiff) > abs(xdiff):
                            check_tile = game_state.map.get_cell(unit.pos.x, unit.pos.y+np.sign(ydiff))
                            if check_tile.citytile == None:
                                if np.sign(ydiff) == 1:
                                    actions.append(unit.move("s"))
                                else:
                                    actions.append(unit.move("n"))
                            else:
                                if np.sign(xdiff) == 1:
                                    actions.append(unit.move("e"))
                                else:
                                    actions.append(unit.move("w"))
                        else:
                            check_tile = game_state.map.get_cell(unit.pos.x+np.sign(xdiff), unit.pos.y)
                            if check_tile.citytile == None:
                                if np.sign(xdiff) == 1:
                                    actions.append(unit.move("e"))
                                else:
                                    actions.append(unit.move("w"))
                            else:
                                if np.sign(ydiff) == 1:
                                    actions.append(unit.move("s"))
                                else:
                                    actions.append(unit.move("n"))
                        continue
                        
                        
                    
                    
                # if unit is a worker and there is no cargo space left, and we have cities, lets return to them
                elif len(player.cities) > 0:
                    if unit.id in unit_to_city_dict and unit_to_city_dict[unit.id] in city_tiles:
                        move_dir = unit.pos.direction_to(unit_to_city_dict[unit.id].pos)
                        actions.append(unit.move(move_dir))
                        
                    else:
                        unit_to_city_dict[unit.id] = get_close_city(player, unit)
                        move_dir = unit.pos.direction_to(unit_to_city_dict[unit.id].pos)
                        actions.append(unit.move(move_dir))
                                
                    closest_city_tile = get_close_city(player, unit)
                    
                    if closest_city_tile is not None:
                        move_dir = unit.pos.direction_to(closest_city_tile.pos)
                        actions.append(unit.move(move_dir))

    # you can add debug annotations using the functions in the annotate object
    # actions.append(annotate.circle(0, 0))
    
    can_create = len(city_tiles) - len(workers)
    
    if len(city_tiles) > 0:
        for city_tile in city_tiles:
            if city_tile.can_act():
                if can_create > 0:
                    actions.append(city_tile.build_worker())
                    can_create -= 1
                else:
                    actions.append(city_tile.research())
            
    
    return actions

In [ ]:
from kaggle_environments import make

env = make("lux_ai_2021", configuration={"width": 12, "height": 12, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run(['agent.py', 'agent.py'])
env.render(mode="ipython", width=800, height=800)

import json
replay = env.toJSON()
with open("replay.json", "w") as f:
    json.dump(replay, f)

## Create a submission
Now we need to create a .tar.gz file with main.py (and agent.py) at the top level. We can then upload this!

In [ ]:
!tar -czf submission.tar.gz *